<a href="https://colab.research.google.com/github/1UC1F3R616/myGoogleCollabNotebooks/blob/master/Image_Similarity_Using_Deep_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Checking if I am using GPU
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.environ['SHARED_DATA_DIR'] = "/content/gdrive/My Drive/SHARED_DATA"
os.environ['IMAGES'] = "/content/gdrive/My Drive/SHARED_DATA/images"

In [4]:
%cd "/content/gdrive/My Drive/SHARED_DATA"

/content/gdrive/My Drive/SHARED_DATA


In [5]:
# -*- coding: utf-8 -*-

# imports
import numpy as np
from skimage import transform
from keras.applications.vgg16 import VGG16
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Embedding
from keras import backend as K

# Decreasing Verbose
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
warnings.filterwarnings('ignore')

def convnet_model_():

    vgg_model = VGG16(weights=None, include_top=False)
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Lambda(lambda  x_: K.l2_normalize(x,axis=1))(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model


def convnet_model():

	vgg_model = VGG16(weights=None, include_top=False)
	x = vgg_model.output
	x - GlobalAveragePooling2D()(x)
	x = Dense(4096, activation='relu')(x)
	x = Dropout(0.6)(x)
	x = Dense(4096, activation='relu')(x)
	x = Dropout(0.6)(x)
	
def deep_rank_model():
 
    convnet_model = convnet_model_()
    first_input = Input(shape=(224,224,3))
    first_conv = Conv2D(96, kernel_size=(8, 8),strides=(16,16), padding='same')(first_input)
    first_max = MaxPool2D(pool_size=(3,3),strides = (4,4),padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(first_max)

    second_input = Input(shape=(224,224,3))
    second_conv = Conv2D(96, kernel_size=(8, 8),strides=(32,32), padding='same')(second_input)
    second_max = MaxPool2D(pool_size=(7,7),strides = (2,2),padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda  x: K.l2_normalize(x,axis=1))(second_max)

    merge_one = concatenate([first_max, second_max])

    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    l2_norm_final = Lambda(lambda  x: K.l2_normalize(x,axis=1))(emb)

    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=l2_norm_final)

    return final_model



def predict_distance(img1, img2):
    """
    High Similarity: Value closer to 0
    Generaly upto 7 we can belive that the image is of same object
    """


    model = deep_rank_model()

    model.load_weights('deepranking-v2-150000.h5')

    image1 = load_img(img1)
    image1 = img_to_array(image1).astype("float64")
    image1 = transform.resize(image1, (224, 224))
    image1 *= 1. / 255
    image1 = np.expand_dims(image1, axis = 0)

    embedding1 = model.predict([image1, image1, image1])[0]

    image2 = load_img(img2) # 'wd1.png' 'b225.png'
    image2 = img_to_array(image2).astype("float64")
    image2 = transform.resize(image2, (224, 224))
    image2 *= 1. / 255
    image2 = np.expand_dims(image2, axis = 0)

    embedding2 = model.predict([image2,image2,image2])[0]

    distance = sum([(embedding1[idx] - embedding2[idx])**2 for idx in range(len(embedding1))])**(0.5)

    return distance


Using TensorFlow backend.


In [6]:
predict_distance('images/d2.jpeg', 'images/d3.jpeg')

0.2107155268793273

In [12]:
# Taking all Images from images directory
# It is known that I am adding only images to images directory but still I have
# used a filter
import glob
image_formats = ['png', 'jpg', 'jpeg']
all_images = []
for image_type in image_formats:
  # all_images += [image.split('/')[-1] for image in glob.glob('images/*.'+image_type)] # To get only names of images, not useful here
  all_images += glob.glob('images/*.'+image_type)
all_images.remove('images/c1.jpg') # size too big, max size keep lower then 300kb
print(all_images)

['images/t1.jpeg', 'images/t2.jpeg', 'images/c2.jpeg', 'images/d1.jpeg', 'images/d2.jpeg', 'images/d3.jpeg']


In [8]:
# from PIL import Image
# image = Image.open('images/c1.jpg')
# print(image.format)
# print(image.mode)
# print(image.size)
# # show the image
# img_resized = image.resize((200,200))
# # report the size of the thumbnail
# print(img_resized.size)

JPEG
RGB
(1800, 1200)
(200, 200)


In [0]:
# for x in all_images:
#   for y in all_images:
#     print('{} - {}: {}'.format(x.split('/')[-1], y.split('/')[-1], predict_distance(x, y)))

"""
Images are getting resized in predict_distance so no need here
"""

# for x in all_images:
#   for y in all_images:
#     im1 = Image.open(x).resize((100, 100))
#     im2 = Image.open(y).resize((100, 100))
#     print('{} - {}: {}'.format(x.split('/')[-1], y.split('/')[-1], predict_distance(im1, im2)))

In [13]:
count = 0
while count != len(all_images):
  # for x in all_images:
  while True:
    x = all_images[count]
    count += 1
    for y in all_images:
      if predict_distance(x, y) < 0.2:
        print('{} - {} : High'.format(x.split('/')[-1], y.split('/')[-1]))
      elif predict_distance(x, y) < 0.5:
        print('{} - {} : Good'.format(x.split('/')[-1], y.split('/')[-1]))
      elif predict_distance(x, y) < 0.89:
        print('{} - {} : Some'.format(x.split('/')[-1], y.split('/')[-1]))
      elif predict_distance(x, y) < 1.2:
        print('{} - {} : Poor'.format(x.split('/')[-1], y.split('/')[-1]))
      elif predict_distance(x, y) < 1.5:
        print('{} - {} : Poorer'.format(x.split('/')[-1], y.split('/')[-1]))
      elif predict_distance(x, y) > 1.5:
        print('{} - {} : Poorest'.format(x.split('/')[-1], y.split('/')[-1]))
    break

t1.jpeg - t1.jpeg : High
t1.jpeg - t2.jpeg : Good
t1.jpeg - c2.jpeg : Poor
t1.jpeg - d1.jpeg : Poor
t1.jpeg - d2.jpeg : Poorest
t1.jpeg - d3.jpeg : Poorest
t2.jpeg - t1.jpeg : Good
t2.jpeg - t2.jpeg : High
t2.jpeg - c2.jpeg : Some
t2.jpeg - d1.jpeg : Poorer


ResourceExhaustedError: ignored